# Cluster Simulation Scheme for Hardware-Efficient VQE#

In [ ]:
# import qiskit and other useful python modules

#plot is directly shown inline
%matplotlib inline 

#import math tools
import numpy as np
# We import plotting tools 
import matplotlib.pyplot as plt 
from   matplotlib import cm
from   matplotlib.ticker import LinearLocator, FormatStrFormatter
# importing Qiskit
import qiskit as qk
import collections
import time
from qiskit.visualization import plot_histogram
from qiskit.providers.aer import noise

#import data IO
import pickle

#import deep copy
import copy


# get simulartor and and real quantum IBMQ backends
# here you need to load your own account
qk.IBMQ.load_account()
provider = qk.IBMQ.get_provider(group='open')

## 16-qubit backend
device_16 = provider.get_backend('ibmq_16_melbourne')
properties = device_16.properties()
noise_model_16 = noise.device.basic_device_noise_model(properties)
basis_gates_16 = noise_model_16.basis_gates
coupling_map_16 = device_16.configuration().coupling_map


## 5-qubit backend
device_5 = provider.get_backend('ibmq_ourense')
properties = device_5.properties()
noise_model_5 = noise.device.basic_device_noise_model(properties)
basis_gates_5 = noise_model_5.basis_gates
coupling_map_5 = device_5.configuration().coupling_map



## Config Parameters and Constants##

In [ ]:
PI = np.pi
NSHOTS = 8000 # number of measurements for each quantum circuit
SEED = int(time.time()) # Random seed for running cicuits

## Input Hamiltonian and Preprocessing

The Hamiltonian file for the molecule BeH2 and classical part of the energies are generated by the file: /Generate BeH2 Hamiltoinian/beh2_reductions.ipynb

The beh2_reductions.ipynb file is a minor revison of the https://github.com/qiskit-community/qiskit-community-tutorials/blob/master/chemistry/beh2_reductions.ipynb. Check https://github.com/qiskit-community/qiskit-community-tutorials for details. 

Our observation is that: measure in Z-basis is good enough to approximate the ground energy of this specific Hamiltonian. This helps reduce the number of circuits need to be run.  


In [ ]:
ham_name='BeH2.txt' #the name of the file
input_file = open(ham_name) 
pauli_list_initial = []
pauli_list = []

##read Hamiltonian from the file
for lines in input_file:
    A = lines.split()[0]
    B = lines.split()[1][1:-4]
    pauli_list_initial.append([A, float(B)])
    
    #Truncation, only measure in Z basis
    flag = 1
    for s in A:
        if (s == 'X' or s == 'Y'):
            flag = 0
    if (flag == 1):
        pauli_list.append([A, float(B)]) 

exact_ground_energy = -15.52877867

# Frozen energy part: -14.684871929146
# Nuclear repulsion energy: 2.6458860546
# energy associated with the operator IIIIII: -2.153967497061597
classical_part = (-14.684871929146) + 2.6458860546 - 2.153967497061597 ##energy can be computed classically
I = np.matrix([[1, 0], [0, 1]])
X = np.matrix([[0, 1], [1, 0]])
Y = np.matrix([[0, -1j], [1j, 0]])
Z = np.matrix([[1, 0], [0, -1]])

#compute the energy of initialized Hamiltonian
H_initial = np.zeros([2**6, 2**6])
for i in range(len(pauli_list_initial)):
    basis = pauli_list_initial[i][0] 
    coef = pauli_list_initial[i][1]
    A = np.matrix(1)
    for s in basis:
        if (s == 'I'):
            A = np.kron(A, I)
        else:
            if (s == 'X'):
                A = np.kron(A, X)
            else:
                if (s == 'Z'):
                    A = np.kron(A, Z)
                else:
                    if (s == 'Y'):
                        A = np.kron(A, Y)
    H_initial = H_initial + A*coef
W, V = np.linalg.eig(H_initial)
idx = np.argsort(W)[0]
print(np.min(W) + classical_part)

##compute the energy of the truncated Hamiltonian
H = np.zeros([2**6, 2**6])
for i in range(len(pauli_list)):
    basis = pauli_list[i][0] 
    coef = pauli_list[i][1]
    A = np.matrix(1)
    for s in basis:
        if (s == 'I'):
            A = np.kron(A, I)
        else:
            if (s == 'X'):
                A = np.kron(A, X)
            else:
                if (s == 'Z'):
                    A = np.kron(A, Z)
                else:
                    if (s == 'Y'):
                        A = np.kron(A, Y)
    H = H + A*coef
W, V = np.linalg.eig(H)
idx = np.argsort(W)[0]
print(np.min(W) + classical_part)

##Compute the evaluation function
weight = np.zeros([2**6])
for idx in range(2**6):
    bit_string = format(idx, '0%db' % 6)
    for (basis, coef) in pauli_list:
        v = 1
        for i in range(len(bit_string)):
            if (basis[i] != 'I'):
                v *= 1-2*int(bit_string[i])
        weight[idx] += v*coef

# Evalutation 

Evaluate the observable in different scenarios

In [ ]:
def evaluate_with_basis(ddict, measurement_basis, coefficient):
    '''
    The function computes the observable value given 
    the measurement results, the measurement basis, and the coefficient
    
    ddict: a dictionary of measurement results, e.g., {'001000': 50}
    measurement_basis: a string representing the mesurement basis, e.g., "XXXYZZ"
    coefficient: the coefficient of the overall observable
    '''
    value = 0
    for bit_string in ddict:
        v = 1
        for i in range(len(bit_string)):
            if (measurement_basis[i] != 'I'):
                v *= 1-2*int(bit_string[i])
        value += v*coefficient*ddict[bit_string] / NSHOTS
    return value

def evaluate(ddict):
    '''
    The function computes the observable value given the measurement results
    by defaut, the measurment is measured with Z basis
    the weight function is precomputed as the coefficient for different measurement results
    '''
    value = 0
    for bit_string in ddict:
        value += weight[int(bit_string, 2)]*ddict[bit_string] / NSHOTS
    return value

def evaluate_with_Hamiltonian(n, depth, param, H):
    '''
    Given the VQE quantum circuit and Hamiltonian, return the associated 'ideal' value of observable 
    
    n: number of qubits
    depth: depth of the circuit (currently we only allow depth = 1)
    param: parameters describing the circuit
    H: Hamiltonian
    '''
    qr = qk.QuantumRegister(n, 'q')
    cr = qk.ClassicalRegister(n, 'c')
    qc = qk.QuantumCircuit(qr, cr)
    
    for qb in range(n):
        qc.rx(param[qb][0], qr[qb])
        qc.rz(param[qb][1], qr[qb])
    
    for d in range(depth):
        
         # pairwise control Z
        for cqb in range(n//2-1):
            qc.cz(qr[cqb],qr[cqb + 1])
        
        for cqb in range(n//2, n-1):
            qc.cz(qr[cqb], qr[cqb+1])
            
        qc.cz(qr[n//2-1], qr[n//2])
            
        # assign parameterized gate
        for qb in range(n):
            qc.rz(param[qb][2+d*3], qr[qb])
            qc.rx(param[qb][3+d*3], qr[qb])
            qc.rz(param[qb][4+d*3], qr[qb])
            

    # Select the StatevectorSimulator from the Aer provider
    simulator = qk.Aer.get_backend('statevector_simulator')

    # Execute and get the resulting state vector
    result = qk.execute(qc, simulator).result()
    vector = result.get_statevector(qc)
    statevector = np.zeros(2**n, dtype='complex')
    for idx in range(2**n):
        bit_string = format(idx, "0%db" % n)
        statevector[int(bit_string[::-1],2)] = vector[idx]
    statevector = np.matrix(statevector)
    # Return the observable 
    return np.trace(statevector*H*statevector.getH())

## Generate the unreduced version of the cirucit

Generate the original version of the cirucit

In [ ]:
# generate the original, unreduced version of the quantum circuit for VQE
def make_VQE_circuit(n, depth, thetas, measurement):
    '''
    n: number of qubits:
    depth: depth of circuits; 
    thetas: the control parameters for VQE; dimension = (n,depth*3+2)
    measurement: measurement basis
    '''
    # construct qubits and classical bits    
    qr = qk.QuantumRegister(n, 'q')
    cr = qk.ClassicalRegister(n, 'c')
    qc = qk.QuantumCircuit(qr, cr)
    
    for qb in range(n):
        qc.rx(thetas[qb][0], qr[qb])
        qc.rz(thetas[qb][1], qr[qb])
    
    for d in range(depth):
        # add barrier
        #qc.barrier(qr)
        
        # pairwise control Z
        for cqb in range(n//2-1):
            qc.cz(qr[cqb],qr[cqb + 1])
        
        for cqb in range(n//2, n-1):
            qc.cz(qr[cqb], qr[cqb + 1])
            
        qc.cz(qr[n//2-1], qr[n//2])
            
        # assign parameterized gate
        for qb in range(n):
            qc.rz(thetas[qb][2+d*3], qr[qb])
            qc.rx(thetas[qb][3+d*3], qr[qb])
            qc.rz(thetas[qb][4+d*3], qr[qb])
        # add barrier
        #qc.barrier(qr)
    
    # measure z
    for qb in range(n):
        if (measurement[qb] == 'X'):
            qc.h(qr[qb])
        if (measurement[qb] == 'Y'):
            qc.sdg(qr[qb])
            qc.h(qr[qb])
        qc.measure(qr[qb],cr[qb])    
    return qc,qr,qc

# obtain sampling distribution from the original, unreduced version of the quantum circuit for VQE
# By default, we run this circuit on the IBM 16-qubit machine or corresponding simulator
def get_orig_VQE_result(n, depth, default_thetas, backend, simulated_noise=False, measurement='ZZZZZZ'):
    '''
    n: number of qubits:
    depth: number of entanglements; 
    default_thetas: the control parameters for VQE; dimension = (n,depth)
    backend: backend the of quantum circuit (simulator or real device)
    simulated_noise: whether adding noise for the simulator
    measurement: measurement basis
    '''
    qc,qr,qc = make_VQE_circuit(n,depth,default_thetas, measurement)
    if (simulated_noise):
        job = qk.execute(qc, backend,shots=NSHOTS, coupling_map=coupling_map_16, noise_model=noise_model_16, 
                     basis_gates=basis_gates_16, 
                    initial_layout={qr[5]: 4, qr[4] : 3, qr[3] : 2, qr[2] : 1, qr[1] : 0, qr[0] : 14})
    else:
        job = qk.execute(qc, backend, shots=NSHOTS, initial_layout={qr[5]: 4, qr[4] : 3, qr[3] : 2, qr[2] : 1, qr[1] : 0, qr[0] : 14})

    result = job.result()
    ddict_tmp = result.get_counts(0)
    
    #reverse the output, index from 0-th qubit 
    ddict = {}
    for bit_string in ddict_tmp:
        ddict[bit_string[::-1]] = ddict_tmp[bit_string]
    return ddict

## Generate the reduced version of the cirucit

Generate the partioned version of the cirucit

In [ ]:
# Compute the probability after we cut the circuit
# This is for handling that IBM real chip cannot do the measurement in the middle
def compute_pre_prob(alpha, beta, gamma):
    '''
    the circuit is  Rz(gamma)Rx(beta)Rz(alpha)
    pre_prob[x][y] is the probability of measuring y when inputing |x>
    '''
    alpha = float(alpha)
    beta = float(beta)
    gamma = float(gamma)
    A = np.matrix([[np.exp(-1j*alpha/2), 0], [0, np.exp(1j*alpha/2)]])
    B = np.matrix([[np.cos(beta/2), -1j*np.sin(beta/2)], [-1j*np.sin(beta/2), np.cos(beta/2)]])              
    C = np.matrix([[np.exp(-1j*gamma/2), 0], [0, np.exp(1j*gamma/2)]])
         
    state0 = np.matrix([[1], [0]])
    state1 = np.matrix([[0], [1]])
    pre_prob = [[0, 0], [0, 0]]
    
    result0 = C*B*A*state0
    pre_prob[0][0] = float(np.abs(result0[0])**2)
    pre_prob[0][1] = float(np.abs(result0[1])**2)
                   
    result1 = C*B*A*state1
    pre_prob[1][0] = float(np.abs(result1[0])**2)
    pre_prob[1][1] = float(np.abs(result1[1])**2)
    
    return pre_prob

In [ ]:
# this is a modifed version of the get_reduced_maxcut_result function for the IBM Q Chips; 
# we have to do this because the current IBMQ devices does not allow gates to occur
# after measurement. This function can work for circuit of depth 1 on real IBM Q chips
def get_real_device_reduced_result(n, depth, thetas, backend, simulated_noise=False):
    '''
        returns the result in n qubit space
        by default the qubit to be disconnected is the last qubit with index n-1
        n: number of qubits:
        depth: number of entanglements; 
        thetas: the control parameters for VQE; dimension = (n,depth*3+2)
        backend: the simulator or the real device
        simulated_noise: adding noise to the simulator
    '''
    # construct container for n-1 qubit results
    final_result = {}
    for idx in range(2 ** n):
        final_result[format(idx,'0%db'% n)] = 0
      
    pre_prob_qubit2 = compute_pre_prob(thetas[2][2], thetas[2][3], thetas[2][4])
    
    pre_prob_qubit3 = compute_pre_prob(thetas[3][2], thetas[3][3], thetas[3][4])
    
    #We suppose n = 6, indexed by 0, 1, 2, 3, 4, 5
    #We cut the circuit at between 2 and 3 
    #enumerate different version of subcircuits 
    for runid in range(6 ** depth):
        # construct qubits and classical bits
        qr = qk.QuantumRegister(n//2, 'q')
        cr = qk.ClassicalRegister(n//2, 'c')
        qc = qk.QuantumCircuit(qr, cr)
        qr2 = qk.QuantumRegister(n//2, 'q')
        cr2 = qk.ClassicalRegister(n//2, 'c')
        qc2 = qk.QuantumCircuit(qr2, cr2) 
      
        #coef for each runid
        coef = 1

        #Add 1-qubit gates
        for qb in range(n//2):
            qc.rx(thetas[qb][0], qr[qb])
            qc.rz(thetas[qb][1], qr[qb])
            
        for qb in range(n//2):
            qc2.rx(thetas[qb+n//2][0], qr2[qb])
            qc2.rz(thetas[qb+n//2][1], qr2[qb])
        
        gate_ver = runid + 1

            # add barrier
        qc.barrier(qr)
        qc2.barrier(qr2)

        #Add 2-qubit gates
        # pairwise control Z
        for cqb in range(n//2-1):
            qc.cz(qr[cqb],qr[cqb + 1])

        for cqb in range(n//2-1):
            qc2.cz(qr2[cqb], qr2[cqb + 1])

        # add the virtual two qubit gate
        qa = qr[2]
        qb = qr2[0]
        if gate_ver == 1:
            coef *= 1/2
        elif gate_ver == 2:
            qc.z(qa)
            qc2.z(qb)
            coef *= 1/2
        elif gate_ver == 3:
            qc2.rz(-np.pi * 1 / 2, qb)

            coef *= -1/2*1
        elif gate_ver == 4:
            qc2.rz(-np.pi * -1 / 2, qb)

            coef *= -1/2*-1
        elif gate_ver == 5:
            qc.rz(-np.pi * 1 / 2, qa)

            coef *= -1/2*1
        elif gate_ver == 6:
            qc.rz(-np.pi * -1 / 2, qa)

            coef *= -1/2*-1

        qc.rz(-np.pi/2, qa)
        qc2.rz(-np.pi/2, qb)

        #Add 1-qubit gates
        for qb in range(n//2):
            if ((gate_ver == 3 or gate_ver == 4) and qb == 2):
                continue
            qc.rz(thetas[qb][2], qr[qb])
            qc.rx(thetas[qb][3], qr[qb])
            qc.rz(thetas[qb][4], qr[qb])

        for qb in range(n//2):
            if ((gate_ver == 5 or gate_ver == 6) and qb == 0):
                continue
            qc2.rz(thetas[qb+n//2][2], qr2[qb])
            qc2.rx(thetas[qb+n//2][3], qr2[qb])
            qc2.rz(thetas[qb+n//2][4], qr2[qb])

        qc.barrier(qr)
        qc2.barrier(qr2)

        # measure z
        for qb in range(n//2):
            qc.measure(qr[qb], cr[qb])
            
        for qb in range(n//2):
            qc2.measure(qr2[qb], cr2[qb])
    
        # execute experiments
        if (simulated_noise):
            job_tmp_1 = qk.execute(qc, backend,shots=NSHOTS, coupling_map=coupling_map_5, noise_model=noise_model_5, 
                     basis_gates=basis_gates_5, 
                    initial_layout={qr[0]: 0, qr[1] : 1, qr[2] : 2})
            
            job_tmp_2 = qk.execute(qc2, backend,shots=NSHOTS, coupling_map=coupling_map_5, noise_model=noise_model_5, 
                     basis_gates=basis_gates_5, 
                    initial_layout={qr2[0]: 0, qr2[1] : 1, qr2[2] : 2})
        else:
            job_tmp_1 = qk.execute(qc, backend, shots=NSHOTS)
            job_tmp_2 = qk.execute(qc2, backend, shots=NSHOTS)
        result_tmp_2 = job_tmp_2.result()
        result_tmp_1 = job_tmp_1.result()
        dict_1 = result_tmp_1.get_counts(0)
        dict_2 = result_tmp_2.get_counts(0)
        
        
        #post-processing the results (linear combination of precalculated coefficients)
        gate_ver = runid + 1
        for key1 in dict_1:
            for key2 in dict_2:
                    
                if gate_ver == 3 or gate_ver == 4:
                    t = int(key1[0])
                    for t_replace in range(2):
                        key = key2 + str(t_replace) + key1[1:]
                        final_result[key] += coef*(1-t*2)*dict_1[key1]*dict_2[key2]*pre_prob_qubit2[t][t_replace]
                    
                if gate_ver == 5 or gate_ver == 6:
                    t = int(key2[-1])
                    for t_replace in range(2):
                        key = key2[:-1] + str(t_replace) + key1
                        final_result[key] += coef*(1-t*2)*dict_1[key1]*dict_2[key2]*pre_prob_qubit3[t][t_replace]
                    
                if gate_ver == 1 or gate_ver == 2:
                    key = key2 + key1
                    final_result[key] += coef*dict_1[key1]*dict_2[key2]
    
    ddict = {}
    #reverse the output, index from 0-th qubit 
    for bit_string in final_result:
        ddict[bit_string[::-1]] = final_result[bit_string] / NSHOTS
    #print(final_result)
    return ddict

# SPSA optimization

In [ ]:
# the simultaneous perturbation stochastic approximation (SPSA) method
# described in https://en.wikipedia.org/wiki/Simultaneous_perturbation_stochastic_approximation 
def SPSA_optimization(n, depth, starting_point, total_test, get_VQE_result, simulated_noise=False, file_name = "params.txt", device=qk.Aer.get_backend('qasm_simulator')):
    '''
    n: number of qubits
    depth: depth of the circuit
    starting_point: for debuging, by default, 0
    total_test: the number of iterations
    get_VQE_result: a function reference for choosing the appropriate algorithms
    simulated_noise: adding noise to the simulator
    file_name: the running data is stored in the file
    device: the simulator or the real quantum chip
    '''
    
    # the following global definitions are in place because
    # we still want to access the last running result
    # in case any error occurs during optimizatrion
    global F
    global Params
    
    L = n*(depth*3+2) # the total number of parameters
    beta = np.ones((L, 1)) #initilize the parameters
    
    if (starting_point > 0): #for debugging, start for a specified parameters path
        total = 0
        for i in range(n):
            for j in range(depth*3+2):
                beta[total] = Params[starting_point][total]
                total = total + 1

    Params = Params[0:starting_point]
    F = F[0:starting_point]
    for T in range(starting_point, total_test):
        
        #the constants for SPSA
        an = 1 / np.power(T+1, 0.3)
        cn = 0.3 / np.power(T+1, 0.5)
        
        #random the gradient estimation direction
        delta = np.random.binomial(1, 0.5, L)*2-1
                
        #store the parameters in the current step
        Params.append(copy.deepcopy(beta))
                
        #evaluate the plus direction
        total = 0 
        for i in range(n):
            for j in range(depth*3+2):
                param[i][j] = beta[total] + delta[total]*cn
                total = total + 1

        ddict = get_VQE_result(n, depth, param, device, simulated_noise)
        v = evaluate(ddict)

        value_plus = float(evaluate_with_Hamiltonian(n, depth, param, H))
        F_plus = v

        #evaluate the minus direction
        total = 0
        for i in range(n):
            for j in range(depth*3+2):
                param[i][j] = beta[total] - delta[total]*cn
                total = total + 1

        ddict = get_VQE_result(n, depth, param, device, simulated_noise)
        v = evaluate(ddict)

        value_minus = float(evaluate_with_Hamiltonian(n, depth, param, H))
        F_minus = v

        #update the parameters
        for i in range(L):
            beta[i] -= an * (F_plus - F_minus) / (2*cn*delta[i])
            
            
        F.append([F_minus, F_plus])
        fw = open(file_name, 'wb')
        pickle.dump([Params, F], fw)

        
        
        print('At step {0}, the F_minus is {1:.5f}, the F_plus is {2:.5f}, and the gradient scale is {3:.5f}\n'.format(T, F_minus, F_plus, an*(F_plus-F_minus)/cn))
        print('At step {0}, the F_real_minus is {1:.5f}, the F_real_plus is {2:.5f}, and the real gradient scale is {3:.5f}\n'.format(T, value_minus, value_plus, an*(value_plus-value_minus)/cn))
        #results = final_result
    return param

### Single shot evalution of the original circuit, paritioned circuit, and ideal observables

In [ ]:
n = 6
depth = 1
np.random.seed(int(time.time()))
SEED = int((time.time()))
param = [[np.random.rand()*5 for j in range(depth*3+2)] for i in range(n)]
NSHOTS = 8000

final_result = get_orig_VQE_result(n, depth, param, qk.Aer.get_backend('qasm_simulator'), simulated_noise = True)
final_result_reduced = get_real_device_reduced_result(n, depth, param, qk.Aer.get_backend('qasm_simulator'), simulated_noise=True)

for idx in range(2 ** n):
    if (not (format(idx, '0%db' % n) in final_result)):
        final_result[format(idx,'0%db'% n)] = 0
    
    if (not (format(idx, '0%db' % n) in final_result_reduced)):
        final_result_reduced[format(idx,'0%db'% n)] = 0
        
from qiskit.visualization import plot_histogram
plot_histogram([final_result_reduced, final_result], legend=["reduced", "original"])

v1 = evaluate(final_result)

v2 = evaluate(final_result_reduced)
        
value = evaluate_with_Hamiltonian(n, depth, param, H_initial)

print(v1, v2, value)

## Run the experiments
The experiment has been run on the machine ibmq_ourense. It may need 3~5 days depend on the server availability.

In [ ]:
n = 6
depth = 1

Params = []
F = []
##reduced 3-qubit circuit
#opt_param = SPSA_optimization(n, depth, 0, 80, get_real_device_reduced_result, simulated_noise=True, file_name='SPSA_result_reduced.txt', device = qk.Aer.get_backend('qasm_simulator'))
opt_param = SPSA_optimization(n, depth, 0, 200, get_real_device_reduced_result, simulated_noise=False, file_name='SPSA_result_reduced.txt', device = provider.get_backend('ibmq_ourense'))


##original 6-qubit circuit
#opt_param = SPSA_optimization(n, depth, 45, 80, get_orig_VQE_result, simulated_noise=True, file_name='SPSA_result_orig.txt', device = qk.Aer.get_backend('qasm_simulator'))
#opt_param = SPSA_optimization(n, depth, 47, 80, get_orig_VQE_result, simulated_noise=False, file_name='SPSA_result_orig.txt', device = provider.get_backend('ibmq_16_melbourne'))